# Mesh Decimation Pipeline

In [1]:
import glob
import os
from tqdm import tqdm

from tqdm.contrib.concurrent import thread_map
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor

import pyvista as pv

C:\Users\tyler\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tyler\Anaconda3\envs\blender341\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\tyler\Anaconda3\envs\blender341\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\tyler\Anaconda3\envs\blender341\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
base_dir = 'Z://Open_data_sets/MICrONS/cubic_mm/'
obj_path = base_dir+ 'objs/'

dec_prcnt = 95
dec_path = base_dir + f'ply_dec_{dec_prcnt}/'
dec_path
os.makedirs(dec_path, exist_ok=True)
N = -1#100


In [6]:
obj_files = glob.glob(f'{obj_path}*.obj')
obj_files[0:10]
os.path.basename(obj_files[0])

'864691131559524855.obj'

In [4]:

files = os.listdir(obj_path)

# Get files with their sizes
files_with_size = [(file, os.path.getsize(os.path.join(obj_path, file))) for file in files]

# Sort files by size in descending order
files_sorted_by_size = sorted(files_with_size, key=lambda x: x[1], reverse=True)

# If you just need file names in sorted order
sorted_files = [file for file, size in files_sorted_by_size]
# print(sorted_files)

existing_files = glob.glob(f'{dec_path}*.ply')

# print(len(obj_files),len(existing_files))
print(len(sorted_files),len(existing_files))




113177 99


In [48]:
obj_files = sorted_files


# Decimation mesh processing and worker functions

In [7]:

def process_obj(filename,dec_prcnt):
    # Create the full path to the .obj file
    file_path = os.path.join(obj_path, filename)
    
    # Load the .obj file
    mesh = pv.read(file_path)
    
    # Perform decimation
    simplified_mesh = mesh.decimate(target_reduction=dec_prcnt/100)  # Reduce to x% of the original mesh
    
    # Save the simplified mesh in PLY format
    output_filename = filename.replace('.obj', '.ply')
    output_path = os.path.join(dec_path, output_filename)
    simplified_mesh.save(output_path)


In [15]:
# Serial processing (working) correctly working progress bar
for file in tqdm(obj_files):
    try:
        filename = os.path.basename(file)
        process_obj(filename, dec_prcnt=dec_prcnt)
    except:
        print(f'Skipping {file} ')

Skipping Z://Open_data_sets/MICrONS/cubic_mm/objs\864691135012428022.obj 


ERROR:root:Line longer than 262144: 
c:\Users\tyler\Anaconda3\envs\blender341\lib\site-packages\pyvista\utilities\fileio.py:198: UserWarning: The VTK reader `vtkOBJReader` in pyvista reader `OBJReader('Z:\Open_data_sets\MICrONS\cubic_mm\objs\864691136534887842.obj')` raised an errorwhile reading the file.
	"Line longer than 262144: "
  warnings.warn(


Skipping Z://Open_data_sets/MICrONS/cubic_mm/objs\864691136534887842.obj 


100%|██████████| 113177/113177 [18:23:23<00:00,  1.71it/s]


In [8]:
# Get the base filenames without extensions
obj_base_names = set(os.path.splitext(os.path.basename(file))[0] for file in obj_files)
existing_base_names = set(os.path.splitext(os.path.basename(file))[0] for file in existing_files)

# Find the difference between the sets to get the files that need processing
files_to_process = obj_base_names - existing_base_names

# Convert back to the full file paths
obj_files_remaining = [os.path.join(obj_path, file + '.obj') for file in files_to_process]

n_samples = len(obj_files_remaining) #139387   

print(n_samples)


113078


In [10]:
len(obj_files_remaining)

113078

# Bulk processing

In [11]:

def worker(file, pbar):
    filename = os.path.basename(file)
    process_obj(filename)
    pbar.update(1)  # Update the progress bar


In [ ]:
# Process remaining serially:

# Serial processing with a correctly working progress bar
for file in tqdm(obj_files_remaining):

    filename = os.path.basename(file)

    # print(filename)

    process_obj(filename)  # Assuming process_obj is defined elsewhere

In [14]:
# Process remaining multithreaded
with ThreadPoolExecutor() as executor, tqdm(total=len(obj_files_remaining)) as pbar:
    # Use map to dispatch the processing tasks
    executor.map(lambda file: worker(file, pbar), obj_files_remaining)



  0%|          | 0/113078 [00:08<?, ?it/s]
